In [26]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np

## 1. Load & Preview Data

In [27]:
# Load the CSV
df = pd.read_csv("../data/Telco_Customer_Churn_cleansed.csv")

# Quick preview
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## 2. 🔧 Feature Engineering

In [28]:
#Chrun by dependents and Partner seemed similar hence the creation of a combined feature
df['HasPartnerOrDependents'] = ((df['Partner'] == 'Yes') | (df['Dependents'] == 'Yes')).astype(int)


In [29]:
#cleaner column names
df = df.rename(columns={'SeniorCitizen': 'IsSeniorCitizen'})


In [30]:
df[['HasPartnerOrDependents', 'IsSeniorCitizen']].head()


,HasPartnerOrDependents,IsSeniorCitizen
0,1,0
1,0,0
2,0,0
3,0,0
4,0,0


### 📝 Justification for `TenureGroup` Feature

Based on the `churn_by_tenure_histogram.png` from the EDA phase, we identified three distinct behavioral patterns among customers at different stages of their lifecycle:

- **New (tenure < 6 months)**: High churn rates likely driven by onboarding or early experience issues.
- **Early (6–18 months)**: Customers still in the decision-making period, where churn remains elevated but begins to stabilize.
- **Loyal (18+ months)**: Customers who are more stable and significantly less likely to churn.

Segmenting tenure in this way enables us to capture potential retention opportunities and improve model interpretability by aligning with real customer lifecycle trends.

In [31]:
def categorize_tenure(tenure):
    if tenure <= 6:
        return 'New'
    elif tenure <= 18:
        return 'Early'
    else:
        return 'Loyal'

df['TenureGroup'] = df['tenure'].apply(categorize_tenure)

tenure_cat_type = CategoricalDtype(categories=['New', 'Early', 'Loyal'], ordered=True)
df['TenureGroup'] = df['TenureGroup'].astype(tenure_cat_type)


### 📝 Rationale for Feature: PackageChangeBehavior
To better understand customer behavior over time, we engineered a feature that compares the expected total charges—calculated as MonthlyCharges × tenure—to the actual TotalCharges recorded. This comparison helps us infer whether a customer has changed their subscription package during their lifecycle:

- If the estimated total charges are significantly higher than the actual charges, it suggests the customer has upgraded their plan at some point.

- If the estimated total charges are lower, it indicates a potential downgrade in service or reduced spending.

- If the difference is within a +5% range, we consider the customer’s billing behavior to be consistent over time.

This feature offers valuable insight into package changes that may be correlated with churn behavior or customer satisfaction.

In [32]:
# Calculate expected Total Charges based on current Monthly Charges and tenure
df['ExpectedTotalCharges'] = df['MonthlyCharges'] * df['tenure']

# Calculate the percentage difference between actual and expected Total Charges
df['BillingDiffPct'] = (df['TotalCharges'] - df['ExpectedTotalCharges']) / df['ExpectedTotalCharges']

# Categorize into behavior groups
def categorize_package_change(diff):
    if diff > 0.05:
        return 'Upgrade'
    elif diff < 0.00:
        return 'Downgrade'
    else:
        return 'Consistent'

df['PackageChangeBehavior'] = df['BillingDiffPct'].apply(categorize_package_change)



In [33]:
df['Churn_num'] = df['Churn'].map({'Yes': 1, 'No': 0})

In [34]:
# One-Hot Encoding categorical variables
categorical_cols = ['gender', 'Partner', 'Dependents', 
                    'PhoneService', 'MultipleLines', 'InternetService', 
                    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                    'TechSupport', 'StreamingTV', 'StreamingMovies', 
                    'Contract', 'PaperlessBilling', 'PaymentMethod','TenureGroup','PackageChangeBehavior']
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

In [35]:
df_encoded.head()

,customerID,IsSeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,HasPartnerOrDependents,ExpectedTotalCharges,BillingDiffPct,Churn_num,...,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,TenureGroup_Early,TenureGroup_Loyal,PackageChangeBehavior_Downgrade,PackageChangeBehavior_Upgrade
0,7590-VHVEG,0,1,29.85,29.85,No,1,29.85,0.000000,0,...,False,False,True,False,True,False,False,False,False,False
1,5575-GNVDE,0,34,56.95,1889.50,No,0,1936.30,-0.024170,0,...,True,False,False,False,False,True,False,True,True,False
2,3668-QPYBK,0,2,53.85,108.15,Yes,0,107.70,0.004178,1,...,False,False,True,False,False,True,False,False,False,False
3,7795-CFOCW,0,45,42.30,1840.75,No,0,1903.50,-0.032966,0,...,True,False,False,False,False,False,False,True,True,False
4,9237-HQITU,0,2,70.70,151.65,Yes,0,141.40,0.072489,1,...,False,False,True,False,True,False,False,False,False,True


In [36]:
df_encoded.to_csv("../data/Telco_Customer_Churn_feature_engineered_v1.csv",index=False)